Broad Steps:

1 - Develop the code to work for a youtube video \
2 - Find a way to iterate over a playlist \
3 - Test-run with a pre-determined playlist \
4 - Develop a general process for selecting and determining which videos to utilize with the program \
5 - Maybe implement/begin feasibility testing of the loop

General Notes:\
1 - When adding to this document, include your name, date and time of the edit you made, and a concise description of \ 1) your thought process/plan of action of how this code will help us parse the videso

In [ ]:
#Sam Liu's code 3/7/2024

#install whisper ai package
!pip install git+https://github.com/openai/whisper.git
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-f7qgi5c1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-f7qgi5c1
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:0

In [ ]:
#Sam Liu's code 3/7/2024

#install pytube package: package performs similar to YouTube API functionalities
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
#ERIC CODE 3/27
#Using keyword search instead of playlist to generate URLs, uses YouTube algorithm to a good job finding high quality vids already

import whisper
import glob
import shutil
from google.colab import files
from pytube import YouTube
from pytube import Playlist

#Import YouTube Data API Client Library
from googleapiclient.discovery import build

#Define YouTube API Key
DEVELOPER_KEY = '[insert here]'
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

#User input
keyword = input("Enter the keyword to search for: ")

#Build YouTube API Data Client
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
#Search for vids with keyword, parameters
search_response = youtube.search().list(
    q=keyword,
    type="video",
    videoSyndicated="any",
    part="id,snippet",
    maxResults=1, #We can change this to whatever
    #videoDuration="short", #have it set to short vids only rn for efficiency
    videoCategoryId="27"  # Education videos only
    ).execute()

#Extract vid IDs from search results, generate links
video_ids = []
for search_result in search_response.get("items", []):
    if search_result["id"]["kind"] == "youtube#video":
        video_ids.append(search_result["id"]["videoId"])
video_urls = [f"https://www.youtube.com/watch?v={video_id}" for video_id in video_ids]

#print to check if all videos in the playlist are properly accessed
print(video_urls)

#clear existing contents from video folder
!rm -rf /content/YouTubeMp4s

for individual_video_url in video_urls:
  try:
    youtube_video_url = YouTube(individual_video_url)
    audio_stream = youtube_video_url.streams.filter(only_audio=True).first()
    audio_stream.download("/content/YouTubeMp4s")
    files.download("/content/YouTubeMp4s")
  except AgeRestrictedError:
    print(f"Skipping age-restricted video: {individual_video_url}")
  except LivestreamError:
    print(f"Skipping livestream video: {individual_video_url}")
    continue

Enter the keyword to search for: Mihaly Csikszentmihalyi: Flow, the secret to happiness
['https://www.youtube.com/watch?v=8h6IMYRoCZw']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#ERIC CODE 3/27
from pathlib import Path

whisper_model = whisper.load_model("base")

#Transcribe all videos from the query and download all .txt files in Files in Colab iteratively
#directory_path is meant to access all the mp4 files in Files, after downloaded every video's audio, we iterate all the audio files and pass them to whisper
directory_path = "/content/YouTubeMp4s/*.mp4"
video_files = glob.glob(directory_path)

#iterate every file in the playlist and transcribe using whispter, later, store them as .txt files
for videos_in_playlist in video_files:
  transcribe_output = whisper_model.transcribe(videos_in_playlist, task="transcribe")
  transcript_file_name = f"/content/YouTubeTxts/{Path(videos_in_playlist).stem}.txt"
  with open(transcript_file_name, "w") as file:
    file.write(str(transcribe_output["text"]))

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Information that is being stored with each youtube video transcript/.txt file:
- Transcript of the text as a .txt file
- URL accessible thorugh the name
- only metadata to include would be the URL in the Chroma DB

In [ ]:
#Sam Liu's code 3/7/2024

from pytube import YouTube
import whisper
from google.colab import files
from pytube import Playlist
import glob

#(1) store one audio from one Youtube Video in Files
#define the url for youtube video to be downloaded
#youtube_video_url = YouTube("https://www.youtube.com/watch?v=yCUg2mIJWHs&list=PLcR22fsoM8Rsi2k4JWwjkKlNo9Q_xSGMr")

#download the youtube video(or videos in a playlist) from the url provided as audio-only file
#audio_stream = youtube_video_url.streams.filter(only_audio=True).first()
#audio_stream.download("/content/YouTube Video downloaded from Pytube Package")

#store the downloaded audio-only file locally in the Files in Colab
#files.download("/content/YouTube Video downloaded from Pytube Package")

#(2) store a playlist of audio from videos in that playlist in Files
#define the url of the playlist
play_list_khan_academy_ap_biology = "https://www.youtube.com/watch?v=juM2ROSLWfw&list=PL0E278E6CA0A3DD5B"

#iterate through the playlist and download all the audio-only files into the Files in Colab
play_list_video_urls = Playlist(play_list_khan_academy_ap_biology)

#print to check if all videos in the playlist are properly accessed
print(play_list_video_urls)

for individual_video_url in play_list_video_urls:
  youtube_video_url = YouTube(individual_video_url)
  audio_stream = youtube_video_url.streams.filter(only_audio=True).first()
  audio_stream.download("/content/YouTube Video downloaded from Pytube Package")
  files.download("/content/YouTube Video downloaded from Pytube Package")



In [ ]:
#Sam Liu's code 3/7/2024

#(1) transcribe one video and download its .txt file in Files in Colab
#call whisper's base model to transcribe the mp4 files stored in "Youtube Video downloaded..."
whisper_model = whisper.load_model("base")
output = whisper_model.transcribe("/content/YouTube Video downloaded from Pytube Package/AP Bio Intro and Themes in Biology.mp4")

#print the transcipt
print(output)

#download the .txt file containing the transcipt to Files in Colab
transcript_file_name = "AP Bio Intro and Themes Transript.txt"
with open(transcript_file_name, "w") as file:
    file.write(str(output))

files.download(transcript_file_name)

#(2) transcribe all videos in the defined playlist and download all .txt files in Files in Colab iteratively
#directory_path is meant to access all the mp4 files in Files, after downloaded every video's audio, we iterate all the audio files and pass them to whisper
directory_path = "/content/YouTube Video downloaded from Pytube Package/*.mp4"
video_files = glob.glob(directory_path)

#iterate every file in the playlist and transcribe using whispter, later, store them as .txt files
for videos_in_playlist in video_files:
  transcribe_output = whisper_model.transcribe(videos_in_playlist)
  transcript_file_names = f"{video_files}.txt"
  with open(transcript_file_names, "w") as file:
    file.write(str(transcribe_output))

